<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%B5%9C%EC%A2%85%EC%98%88%EC%B8%A1/%EB%8C%80%EB%B6%84%EB%A5%98_%EC%98%88%EC%B8%A13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 129.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=f3f987920b5b4ed65f5e0e971143c89d8c41a81c648aae171b5cd6ed7bc1bfbc
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 7.3 MB/s 
     |████████████████████████████████| 769 kB 7.4 MB/s 
     |████████████████████████████████| 3.0 MB 90.2 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-a8fryv50
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_G_inc.csv		   df_P_inc.csv
'Colab Notebooks'	 df_H_inc_노이.csv	  'My Drive'
 C_ori_model.pt		 df_J_inc_again_노이.csv   대분류모델.pt
'답안 작성용 파일.csv'	 df_J_inc.csv		   Q_inc_model.pt
 df_C_inc.csv		 df_J_inc_dain.csv	   spell_check
'# df_G_inc.csv'	 df_K_inc.csv		   ts_spellcheck.csv


In [4]:
import pandas as pd
ts=pd.read_csv('/content/gdrive/MyDrive/ts_spellcheck.csv' ,encoding = "utf-8")

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [6]:
model = torch.load('/content/gdrive/MyDrive/대분류모델.pt')

model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [7]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
ts1=ts[:20000]
ts2=ts[20000:40000]
ts3=ts[40000:60000]
ts4=ts[60000:80000]
ts5=ts[80000:]

ts1=ts1.reset_index(drop=True)
ts2=ts2.reset_index(drop=True)
ts3=ts3.reset_index(drop=True)
ts4=ts4.reset_index(drop=True)
ts5=ts5.reset_index(drop=True)

In [9]:
result=[]
for sentence in ts3['text']:
  temp=predict(sentence)
  result.append(temp)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [10]:
a=pd.DataFrame(result, columns=['digit_1'])
a.loc[(a['digit_1'] == 0), 'digit_1'] = "A"
a.loc[(a['digit_1'] == 1), 'digit_1'] = "B" 
a.loc[(a['digit_1'] == 2), 'digit_1'] = "C"
a.loc[(a['digit_1'] == 3), 'digit_1'] = "D" 
a.loc[(a['digit_1'] == 4), 'digit_1'] = "E" 
a.loc[(a['digit_1'] == 5), 'digit_1'] = "F" 
a.loc[(a['digit_1'] == 6), 'digit_1'] = "G" 
a.loc[(a['digit_1'] == 7), 'digit_1'] = "H"
a.loc[(a['digit_1'] == 8), 'digit_1'] = "I" 
a.loc[(a['digit_1'] == 9), 'digit_1'] = "J" 
a.loc[(a['digit_1'] == 10), 'digit_1'] = "K" 
a.loc[(a['digit_1'] == 11), 'digit_1'] = "L"
a.loc[(a['digit_1'] == 12), 'digit_1'] = "M" 
a.loc[(a['digit_1'] == 13), 'digit_1'] = "N" 
a.loc[(a['digit_1'] == 14), 'digit_1'] = "O" 
a.loc[(a['digit_1'] == 15), 'digit_1'] = "P"
a.loc[(a['digit_1'] == 16), 'digit_1'] = "Q" 
a.loc[(a['digit_1'] == 17), 'digit_1'] = "R" 
a.loc[(a['digit_1'] == 18), 'digit_1'] = "S" 

In [11]:
ts3['digit_1']=a

In [12]:
ts3.to_csv('result_대분류3.csv', index=False, encoding='utf-8-sig')